In [21]:
import psycopg2
import pandas as pd

In [22]:
# https://docs.flashbots.net/flashbots-data/mev-inspect-py/overview

In [23]:
username = ! kubectl get secret mev-inspect-db-credentials -o jsonpath="{.data.username}" | base64 --decode
password = ! kubectl get secret mev-inspect-db-credentials -o jsonpath="{.data.password}" | base64 --decode

param_dic = {
    "host"      : "localhost",
    "database"  : "mev_inspect",
    "user"      : username[0],
    "password"  : password[0],
    "port"      : "5432"
}

conn = psycopg2.connect(
    database    =param_dic['database'], 
    user        =param_dic['user'], 
    password    =param_dic['password'], 
    host        =param_dic['host'], 
    port        =param_dic['port']
)

In [24]:
cur = conn.cursor()
cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public'")
table_names = cur.fetchall()
table_names = [name_tuple[0] for name_tuple in table_names]
cur.close()

In [25]:
blocks = pd.read_sql_query('select * from blocks',con=conn)

In [26]:
blocks

,block_number,block_timestamp
0,12914947.0,2021-07-28 14:29:44
1,12914949.0,2021-07-28 14:30:17
2,12914944.0,2021-07-28 14:28:24
3,12914946.0,2021-07-28 14:29:36
4,12914945.0,2021-07-28 14:29:30
...,...,...
7053,13774095.0,2021-12-10 00:04:25
7054,13774092.0,2021-12-10 00:04:17
7055,13774099.0,2021-12-10 00:06:01
7056,13774097.0,2021-12-10 00:04:55


In [20]:
blocks[(blocks['block_timestamp'] >= '2021-12-10') & (blocks['block_timestamp'] < '2021-12-11')].sort_values(['block_timestamp'])

,block_number,block_timestamp
7032,13774075.0,2021-12-10 00:00:22
7033,13774076.0,2021-12-10 00:00:24
7035,13774077.0,2021-12-10 00:00:30
7036,13774078.0,2021-12-10 00:00:52
7043,13774079.0,2021-12-10 00:00:55
...,...,...
5418,13780497.0,2021-12-10 23:57:59
5415,13780498.0,2021-12-10 23:58:20
5419,13780499.0,2021-12-10 23:58:43
5420,13780500.0,2021-12-10 23:59:32


In [27]:
table_names

['transfers',
 'blocks',
 'alembic_version',
 'classified_traces',
 'arbitrages',
 'arbitrage_swaps',
 'swaps',
 'miner_payments',
 'latest_block_update',
 'liquidations',
 'punk_snipes',
 'punk_bids',
 'punk_bid_acceptances',
 'prices']

In [28]:
table_names.remove('alembic_version')
table_names.remove('classified_traces')

In [29]:
table_names

['transfers',
 'blocks',
 'arbitrages',
 'arbitrage_swaps',
 'swaps',
 'miner_payments',
 'latest_block_update',
 'liquidations',
 'punk_snipes',
 'punk_bids',
 'punk_bid_acceptances',
 'prices']

In [63]:
for table_name in table_names:
    df = pd.read_sql_query('select * from {}'.format(table_name), con=conn)
    df.to_csv('../../data/mev_inspect/{}.csv'.format(table_name), index=False)

In [30]:
conn.close()